## 시험설명 
- 시험시간 : 3시간 (180분)
- 시험유형 : 총 6문제

__제 2유형__
- 1문제 (40점)
- 주제 : 데이터 분석(분류/회귀/비지도학습)    

    

__데이터 분석 순서__
1. 라이브러리 및 데이터확인
2. 데이터탐색(EDA)
3. 데이터 전처리 및 분리
4. 모델링 및 성능평가
5. 예측값 제출 

#### 1. 암기할 클래스  및 라이브러리 모음

##### 표준화 정규화

In [ ]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler

##### 데이터 분리

In [ ]:
from sklearn.model_selection import train_test_split 

##### 모델 선정

In [ ]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

##### 모델 성능 평가관련 
- 분류 모델 성능평가 : 정확도, 정밀도, 재현율, f1_score 
- roc_auc_score : 이진분류일때만 사용가능! 

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, recall_score,precision_score, roc_auc_score

- 회귀 모델 성능평가 : MSE, RMSE, MAE, MAPE, r2_score
- MSE(평균 제곱 오차 )
- RMSE(평균 제곱근 오차) : MSE에 루트 씌운 값 
- MAE(평균 절대 오차): (실제값 - 예측값)의 절댓값의 평균
- MAPE(평균 절대비율 오차) : MSE, RMSE 단점 보완 
- r2_score(결정계수)

In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

##### 01. 데이터 탐색 (EDA)

In [ ]:
# 데이터 행렬 확인
df.shape()

# 초기 데이터 확인 
df.head()

# 변수명과 데이터 타입이 매칭되는지, 결측치 있는지 확인
df.info()

# 결측치 확인 
df.isnull().sum()

# 데이터 형태 확인 
df.dtypes

# 각 카테고리별 종류 확인 
df['컬럼1'].value_counts()

# 연속값 - 이상치 확인
df.describe().T

# 범주형 데이터 - include='category' | 'object'
df.describe(include='category').T

# cust_id 추가 하기
x_train.rename(columns={'index':'cust_id'},inplace=True)
x_test.rename(columns={'index':'cust_id'},inplace=True)

# 아래두개 같은 코드 
y_train.columns = ({'cust_id','target'})
y_train.columns = ['cust_id','target']

##### 02. 데이터 전처리 및 분리 
- 결측치 처리
    - 결측치 제거
    - 결측치 대체
- 이상치
    - 이상치 제거
    - 이상치 대체
- 변수처리 
    - 불필요한 변수 제거
    - 필요시 변수 추가 (파생변수 생성)
    - 원핫 인코딩(범주형 데이터)

결측치 처리 

In [ ]:
# 결측치 제거 
#   - 행을 기준으로 결측치가 있는 행을 삭제 한다 
df = df.dropna()

# 결측치 대체 (평균값, 중앙값, 최빈값)
#   - 연속형의 경우 이상값에 영향이 가장 적은 중앙값 활용
#   - 범주형의 경우 최빈값  활용 

# 연속형 
col1_median = df['col1'].median()
df['컬럼1'].fillna(col1_median)

# 범주형
col1_mode = df['col1'].mode()
df['컬럼1'].fillna(col1_mode)

이상치 처리 

In [ ]:
# 이상치 제거 
# - 이상치 확인 (IQR로 확인)
Q1 = df['col1'].quantile(0.25)
Q3 = df['col1'].quantile(0.75)
IQR = abs(Q1-Q3)

upper = Q3 + 1.5*IQR
lower = Q1 - 1.5*IQR

cond1 = df['col1'] <= upper 
cond2 = df['col1'] >= lower

df = df[cond1 & cond2] 


In [ ]:
# 이상치 대체 (최대값 or 최솟값으로 대체)

# 예시 - 최대값 대체 
cond1 = (df['col1']<=10)
col1_max = df[cond1]['col'].max()

# df 데이터 프레임의 col1 컬럼 값이 10보다 크면 col1_max 값으로, 그렇지 않으면 col1 값 그대로 유지 
df['col1'] = np.where(df['col1']>=10, col1_max,df['col1'])


변수 처리 


In [ ]:
# 불필요한 변수 제거
방법 1. df = df.drop(columns = ['변수1','변수2'])
방법 2. df = df.drop(['변수1','변수2'],axis=1)

# 필요시 변수 추가 (파생변수 생성)
df['파생변수'] = df['A']+df['B'] (파생변수 생성 형태)

# 원핫인코딩 (가변수 처리) : 범주형 데이터 다룰때
x_train = pd.get_dummies(x_train)
x_test = pd.get_dummies(x_test)
y_train = pd.get_dummies(y_train)

# 원핫인코딩 후 feature 변수가 다른 경우
# reindex 활용

x_train = x_train.reindex(columns = x_test.columns, fill_value=0)

##### 03. 데이터분리

In [ ]:
# 회귀모델
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train['col1'],test_size=0.2, randomstate=2024)

# 분류모델 : 계층적 데이터 추출옵션 사용 -> stratify=y_trian['col1']
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train['col1'],test_size=0.2,stratify=y_train['col1'],random_state=2024)

##### 04. 모델링 및 성능평가

In [ ]:
# cust_id 변수 처리 하기 
# cust_id는 모델에 반영이 되면 안된다 
cust_id = x_test['cust_id'].copy()
x_train = x_train.drop(columns=['cust_id'],axis=1)
x_test = x_test.drop(columns=['cust_id'], axis=1)


회귀모델 - 생성 및 예측

In [ ]:
model = RandomForestRegressor()
model.fit(x_train,y_train)

# 모델을 이용한 테스트 데이터 예측 
y_pred = model.predict(x_val)

분류모델 - 생성 및 예측

In [ ]:
model = RandomForestClassifier()
model.fit(x_train,y_train)

# 모델을 이용한 테스트 데이터 예측 
y_pred = model.predict(x_val)

성능평가 - 회귀모델

In [ ]:
mse = mean_squared_error(y_val,y_pred) # 순서 주의! 실제값, 예측값
rsq = r2_score(y_val,y_pred) # 순서 주의! 실제값, 예측값 
RSME = mse**2


성능평가 - 분류모델

In [ ]:
accuracy = accuracy_score(y_val, y_pred)
f1_s = f1_score(y_val,y_pred)

# roc_auc_score은 이진분류일때만 사용 가능 
roc_auc = roc_auc_score(y_val,y_pred)

##### 05. 예측값 제출

In [ ]:
y_result = model.predict(x_test)

# 미리 추출해둔 cust_id랑 합치기 
result = pd.DataFrame({'cust_id' : cust_id , 'result' : y_result})

# 저장 
result.to_csv('test_result.csv', index=False)